# Chapter 5: Querying Multiple Tables

As seen in the previous chapters, joining tables is often or primarily required for information gathering

In [1]:
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, URL, select, func
from sqlalchemy.orm import Session
import pandas as pd

from utils import print_sql_statement


load_dotenv()

url_object = URL.create(
    os.environ["DB_ENGINE"],
    username=os.environ["DB_USER"],
    password=os.environ["DB_PASSWD"],
    host=os.environ["DB_HOST"],
    database=os.environ["DB_NAME"],
)

engine = create_engine(url_object)

# Joining Three Tables

Find all the accounts opened by 'experienced' (hired prior to 2003) tellers currently assigned to the 'Woburn Branch'

This is a little complicated, but the logic is
    * Get all accounts
    * Join all accounts with all the 'experienced' tellers
    * Join all experienced tellers at the Woburn branch

In [22]:
from datetime import date

from sqlalchemy import and_
from sqlalchemy.orm import aliased

from model import Account, Branch, Employee


with Session(engine) as session:
    df = pd.read_sql_query(
        """
        SELECT
            a.account_id
            , a.cust_id
            , a.open_date
            , a.product_cd
        FROM
            account a
        -- ASSOICATE ALL EXPERIENCED TELLERS WITH ACCOUNTS
        JOIN (
            SELECT
                emp_id
                , assigned_branch_id
            FROM
                employee
            WHERE
                employee.start_date <= '2003-01-01'
                AND
                employee.title LIKE '%Teller%'
        ) e ON e.emp_id = a.open_emp_id
        -- ASSOCIATED ALL EMPLOYEES AT THE WOBURN BRANCH
        JOIN (
            SELECT
                branch_id
            FROM
                branch
            WHERE
                branch.name = 'Woburn Branch'
        ) b ON b.branch_id = e.assigned_branch_id
        ;
        """,
        con=session.connection()
    )

    # Get the employee sub-query
    employee_subquery = (
        select(
            Employee.emp_id,
            Employee.assigned_branch_id
        )
        .select_from(Employee)
        .where(
            and_(
                Employee.title.like("%Teller%"),
                Employee.start_date <= date(2003, 1, 1)
            )
        )
        .subquery("e")
    )
    employee_alias: Employee = aliased(Employee, employee_subquery)

    # Get the branch sub-query
    branch_subquery = (
        select(
            Branch.branch_id
        )
        .select_from(Branch)
        .where(Branch.name == "Woburn Branch")
        .subquery("b")
    )
    branch_alias: Branch = aliased(Branch, branch_subquery)

    statement = (
        # Get all accounts
        select(
            Account.account_id,
            Account.cust_id,
            Account.open_date,
            Account.product_cd
        )
        .select_from(Account)
        # Associate all accounts with the experienced teller
        .join(
            employee_subquery,
            Account.open_emp_id == employee_alias.emp_id
        )
        # Associated all the tellers at the Woburn Branch
        .join(
            branch_subquery,
            branch_alias.branch_id == employee_alias.assigned_branch_id
        )
    )
    print_sql_statement(statement)
    results = session.execute(statement).all()

print(df)
print(results)

"""SELECT account.account_id, account.cust_id, account.open_date, account.product_cd 
FROM account JOIN (SELECT employee.emp_id AS emp_id, employee.assigned_branch_id AS assigned_branch_id 
FROM employee 
WHERE employee.title LIKE :title_1 AND employee.start_date <= :start_date_1) AS e ON account.open_emp_id = e.emp_id JOIN (SELECT branch.branch_id AS branch_id 
FROM branch 
WHERE branch.name = :name_1) AS b ON b.branch_id = e.assigned_branch_id"""
   account_id  cust_id   open_date product_cd
0           1        1  2000-01-15        CHK
1           2        1  2000-01-15        SAV
2           3        1  2004-06-30         CD
3           4        2  2001-03-12        CHK
4           5        2  2001-03-12        SAV
5          17        7  2004-01-12         CD
6          27       11  2004-03-22        BUS
[(1, 1, datetime.date(2000, 1, 15), 'CHK'), (2, 1, datetime.date(2000, 1, 15), 'SAV'), (3, 1, datetime.date(2004, 6, 30), 'CD'), (4, 2, datetime.date(2001, 3, 12), 'CHK'), (5, 2, 

# Repeated Table Joins

List all checking (product_cd == 'CHK') account IDs with the account opening branch, opening employee, and the opening employee's currently assigned branch.

The branch ID is a foreign key in both employee and account. But we need to use the branch table in both. So we need to use two different aliases for the branch table.

In [42]:

with Session(engine) as session:
    df = pd.read_sql_query(
        """
        SELECT
            a.account_id account_id
            , e.emp_id emp_id
            , b_a.name open_branch
            , b_e.name emp_branch
        FROM
            account
        a JOIN (
            SELECT
                branch_id
                , name
            FROM branch
        )
        b_a ON b_a.branch_id = a.open_branch_id
        JOIN (
            SELECT
                emp_id
                , assigned_branch_id
            FROM
                employee
        )
        e ON e.emp_id = a.open_emp_id
        JOIN (
            SELECT
                branch_id
                , name
            FROM
                branch
        )
        b_e ON b_e.branch_id = e.assigned_branch_id
        WHERE
            a.product_cd = 'CHK'
        ORDER BY e.emp_id
        ;
        """,
        con=session.connection()
    )

    employee_subquery = (
        select(
            Employee.emp_id,
            Employee.assigned_branch_id
        )
        .select_from(Employee)
    ).subquery("e")
    employee_alias: Employee = aliased(Employee, employee_subquery)

    branch_for_account_subquery = (
        select(
            Branch.branch_id,
            Branch.name
        )
        .select_from(Branch)
    ).subquery("b_a")
    branch_for_account_alias: Branch = aliased(Branch, branch_for_account_subquery)

    branch_for_employee_subquery = (
        select(
            Branch.branch_id,
            Branch.name
        )
        .select_from(Branch)
    ).subquery("b_e")
    branch_for_employee_alias: Branch = aliased(Branch, branch_for_employee_subquery)

    statement = (
        select(
            Account.account_id.label("account_id"),
            employee_alias.emp_id.label("emp_id"),
            branch_for_account_alias.name.label("open_branch"),
            branch_for_employee_alias.name.label("emp_branch")
        )
        .select_from(Account)
        .join(
            employee_subquery,
            employee_alias.emp_id == Account.open_emp_id
        )
        .join(
            branch_for_account_subquery,
            branch_for_account_alias.branch_id == Account.open_branch_id
        )
        .join(
            branch_for_employee_subquery,
            branch_for_employee_alias.branch_id == employee_alias.assigned_branch_id
        )
        .where(Account.product_cd == "CHK")
        .order_by(employee_alias.emp_id)
    )
    print_sql_statement(statement)
    results = session.execute(statement).all()

print(df)
print(results)

"""SELECT account.account_id AS account_id, e.emp_id AS emp_id, b_a.name AS open_branch, b_e.name AS emp_branch 
FROM account JOIN (SELECT employee.emp_id AS emp_id, employee.assigned_branch_id AS assigned_branch_id 
FROM employee) AS e ON e.emp_id = account.open_emp_id JOIN (SELECT branch.branch_id AS branch_id, branch.name AS name 
FROM branch) AS b_a ON b_a.branch_id = account.open_branch_id JOIN (SELECT branch.branch_id AS branch_id, branch.name AS name 
FROM branch) AS b_e ON b_e.branch_id = e.assigned_branch_id 
WHERE account.product_cd = :product_cd_1 ORDER BY e.emp_id"""
   account_id  emp_id    open_branch     emp_branch
0          10       1   Headquarters   Headquarters
1          14       1   Headquarters   Headquarters
2          21       1   Headquarters   Headquarters
3           1      10  Woburn Branch  Woburn Branch
4           4      10  Woburn Branch  Woburn Branch
5           7      13  Quincy Branch  Quincy Branch
6          13      16  So. NH Branch  So. NH Branc